In [1]:
from gurobi import *
import numpy as np
import pandas as pd
from scipy.spatial import distance
from itertools import chain, combinations

# PDSTSP


In [2]:
class Data:
    def __init__(self):
        self.customerNum = 0 
        self.nodeNum     = 0 
        self.droneNum    = 2
        self.cities      = []
        self.cor_X       = [] 
        self.cor_Y       = [] 
        self.serviceTime = [] 
        self.disMatrix   = [[]]
        self.dt          = None
        self.i_pot = None
        self.cus_can_served_by_drone = None
        self.drone_distances = None
        self.truck_distances = None
        
      
        

    def readData(self, path):
        self.dt = pd.read_csv(path, header = None).to_numpy()[:-1]
        self.customerNum = len(self.dt)
        self.i_pot = self.dt[0, 1:3]
        self.nodeNum = self.customerNum + 2 
        
        self.cities = [self.dt[i, 0] for i in range(len(self.dt))]
        
        self.cus_can_served_by_drone = [i for i in range(len(self.dt)) if self.dt[i, 3] == 0]
        
        self.drone_distances = [round(distance.euclidean((self.dt[i, 1:3]), self.i_pot),2)
                                if self.dt[i, 3] == 0 else float('inf')
                                for i in range(len(self.dt))]
        self.truck_distances = [[round(distance.cityblock(self.dt[i, 1:3], self.dt[j, 1:3]),1)
                                 for i in range(len(self.dt))] for j in range(len(self.dt))]
    
    

        
       


In [3]:
data = Data()

data.readData("20140813T112010.csv")
print(data.dt)

print(data.customerNum)

[[ 0.   0.   0.   0.4]
 [ 1.   1.7  3.9  0. ]
 [ 2.   3.4  0.1  0. ]
 [ 3.   1.2  5.2  1. ]
 [ 4.   9.3  3.4  0. ]
 [ 5.   2.9  1.9  0. ]
 [ 6.   0.2  5.1  0. ]
 [ 7.   5.7  0.1  0. ]
 [ 8.   1.3 10.8  0. ]
 [ 9.   1.4  0.8  0. ]]
10


In [4]:
print(data.dt[1, 1:3])

[1.7 3.9]


In [5]:
print(data.cities)

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]


In [6]:
print(data.truck_distances)

[[0.0, 5.6, 3.5, 6.4, 12.7, 4.8, 5.3, 5.8, 12.1, 2.2], [5.6, 0.0, 5.5, 1.8, 8.1, 3.2, 2.7, 7.8, 7.3, 3.4], [3.5, 5.5, 0.0, 7.3, 9.2, 2.3, 8.2, 2.3, 12.8, 2.7], [6.4, 1.8, 7.3, 0.0, 9.9, 5.0, 1.1, 9.6, 5.7, 4.6], [12.7, 8.1, 9.2, 9.9, 0.0, 7.9, 10.8, 6.9, 15.4, 10.5], [4.8, 3.2, 2.3, 5.0, 7.9, 0.0, 5.9, 4.6, 10.5, 2.6], [5.3, 2.7, 8.2, 1.1, 10.8, 5.9, 0.0, 10.5, 6.8, 5.5], [5.8, 7.8, 2.3, 9.6, 6.9, 4.6, 10.5, 0.0, 15.1, 5.0], [12.1, 7.3, 12.8, 5.7, 15.4, 10.5, 6.8, 15.1, 0.0, 10.1], [2.2, 3.4, 2.7, 4.6, 10.5, 2.6, 5.5, 5.0, 10.1, 0.0]]


In [7]:
print(data.drone_distances)

[inf, 4.25, 3.4, inf, 9.9, 3.47, 5.1, 5.7, 10.88, 1.61]


In [8]:
model = Model("PDSTSP")

Academic license - for non-commercial use only - expires 2021-07-04
Using license file /home/quanghuy205/gurobi.lic


In [9]:
#SET
V = [i for i in range(data.customerNum)]
C = [i for i in range (1, data.customerNum)]
U = [k for k in range(data.droneNum)]

C_U = data.cus_can_served_by_drone


C_truck = {(i,j): data.truck_distances[i][j] for i in V for j in V}
C_drone = data.drone_distances

#Decision variables

# x_ij if (i->j) in vehicle tour
# # y_im = 1 if cus i assigned to drone m ()

x = [[[] for i in V] for j in V]  
y = [[[] for i in V] for k in U] 
x

[[[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []]]

In [24]:
#1
#completion time
alpha = model.addVar(0, GRB.INFINITY, 1.0, GRB.CONTINUOUS, "traveltime")
model.update()
expr = LinExpr(0)
expr.addTerms(1.0, alpha)

model.setObjective(expr, GRB.MINIMIZE)
expr.clear()

In [25]:
#2
expr = LinExpr(0)
for i in V:
    for j in V:
#         if i != j:
        x[i][j] = model.addVar(0, 1, vtype = GRB.BINARY,name = "x%d,%d" %(i,j))
                
        model.update()
        expr.addTerms(data.truck_distances[i][j], x[i][j])
#         else:
#              x[i][i] = model.addVar(0.0, 0.0, 0.0, GRB.BINARY, "x%d,%d" %(i,j))
                
print(expr)

model.addConstr(alpha >= expr, "truckTime")
expr.clear()
model.update()

<gurobi.LinExpr: 0.0 x0,0 + 5.6 x0,1 + 3.5 x0,2 + 6.4 x0,3 + 12.7 x0,4 + 4.8 x0,5 + 5.3 x0,6 + 5.8 x0,7 + 12.1 x0,8 + 2.2 x0,9 + 5.6 x1,0 + 0.0 x1,1 + 5.5 x1,2 + 1.8 x1,3 + 8.1 x1,4 + 3.2 x1,5 + 2.7 x1,6 + 7.8 x1,7 + 7.3 x1,8 + 3.4 x1,9 + 3.5 x2,0 + 5.5 x2,1 + 0.0 x2,2 + 7.3 x2,3 + 9.2 x2,4 + 2.3 x2,5 + 8.2 x2,6 + 2.3 x2,7 + 12.8 x2,8 + 2.7 x2,9 + 6.4 x3,0 + 1.8 x3,1 + 7.3 x3,2 + 0.0 x3,3 + 9.9 x3,4 + 5.0 x3,5 + 1.1 x3,6 + 9.6 x3,7 + 5.7 x3,8 + 4.6 x3,9 + 12.7 x4,0 + 8.1 x4,1 + 9.2 x4,2 + 9.9 x4,3 + 0.0 x4,4 + 7.9 x4,5 + 10.8 x4,6 + 6.9 x4,7 + 15.4 x4,8 + 10.5 x4,9 + 4.8 x5,0 + 3.2 x5,1 + 2.3 x5,2 + 5.0 x5,3 + 7.9 x5,4 + 0.0 x5,5 + 5.9 x5,6 + 4.6 x5,7 + 10.5 x5,8 + 2.6 x5,9 + 5.3 x6,0 + 2.7 x6,1 + 8.2 x6,2 + 1.1 x6,3 + 10.8 x6,4 + 5.9 x6,5 + 0.0 x6,6 + 10.5 x6,7 + 6.8 x6,8 + 5.5 x6,9 + 5.8 x7,0 + 7.8 x7,1 + 2.3 x7,2 + 9.6 x7,3 + 6.9 x7,4 + 4.6 x7,5 + 10.5 x7,6 + 0.0 x7,7 + 15.1 x7,8 + 5.0 x7,9 + 12.1 x8,0 + 7.3 x8,1 + 12.8 x8,2 + 5.7 x8,3 + 15.4 x8,4 + 10.5 x8,5 + 6.8 x8,6 + 15.1 x8,7 

In [12]:
#3
for k in U:
    expr = LinExpr(0)
    for i in C:
        
        if i in C_U:
            y[k][i] = model.addVar(0,1, vtype= GRB.BINARY, name =  "y%d,%d" %(k,i))
            print(i)
        
            expr.addTerms(data.drone_distances[i], y[k][i])
        else:
            y[k][i] = model.addVar(0, 0, vtype = GRB.BINARY, name = "y%d,%d" %(k,i))
    model.update()
    print(expr)
    model.addConstr(alpha >= expr, "dronetime")
    expr.clear()
expr.clear()    


1
2
4
5
6
7
8
9
<gurobi.LinExpr: 4.25 y0,1 + 3.4 y0,2 + 9.9 y0,4 + 3.47 y0,5 + 5.1 y0,6 + 5.7 y0,7 + 10.88 y0,8 + 1.61 y0,9>
1
2
4
5
6
7
8
9
<gurobi.LinExpr: 4.25 y1,1 + 3.4 y1,2 + 9.9 y1,4 + 3.47 y1,5 + 5.1 y1,6 + 5.7 y1,7 + 10.88 y1,8 + 1.61 y1,9>


In [13]:
#4
for j in C:
    
    expr1 = LinExpr(0)
    expr2 = LinExpr(0)
    
    for i in V:
            
            expr1.addTerms(1.0, x[i][j])
    print(expr1)
    
    if j in C_U:
        for k in U:
            expr2.addTerms(1.0, y[k][j])
    
    print(expr2)
    print("------------------")
    
    model.addConstr(expr1 + expr2 == 1, "served customer once")
    expr1.clear()
    expr2.clear()
expr1.clear()
expr2.clear()

<gurobi.LinExpr: x0,1 + x1,1 + x2,1 + x3,1 + x4,1 + x5,1 + x6,1 + x7,1 + x8,1 + x9,1>
<gurobi.LinExpr: y0,1 + y1,1>
------------------
<gurobi.LinExpr: x0,2 + x1,2 + x2,2 + x3,2 + x4,2 + x5,2 + x6,2 + x7,2 + x8,2 + x9,2>
<gurobi.LinExpr: y0,2 + y1,2>
------------------
<gurobi.LinExpr: x0,3 + x1,3 + x2,3 + x3,3 + x4,3 + x5,3 + x6,3 + x7,3 + x8,3 + x9,3>
<gurobi.LinExpr: 0.0>
------------------
<gurobi.LinExpr: x0,4 + x1,4 + x2,4 + x3,4 + x4,4 + x5,4 + x6,4 + x7,4 + x8,4 + x9,4>
<gurobi.LinExpr: y0,4 + y1,4>
------------------
<gurobi.LinExpr: x0,5 + x1,5 + x2,5 + x3,5 + x4,5 + x5,5 + x6,5 + x7,5 + x8,5 + x9,5>
<gurobi.LinExpr: y0,5 + y1,5>
------------------
<gurobi.LinExpr: x0,6 + x1,6 + x2,6 + x3,6 + x4,6 + x5,6 + x6,6 + x7,6 + x8,6 + x9,6>
<gurobi.LinExpr: y0,6 + y1,6>
------------------
<gurobi.LinExpr: x0,7 + x1,7 + x2,7 + x3,7 + x4,7 + x5,7 + x6,7 + x7,7 + x8,7 + x9,7>
<gurobi.LinExpr: y0,7 + y1,7>
------------------
<gurobi.LinExpr: x0,8 + x1,8 + x2,8 + x3,8 + x4,8 + x5,8 + x6,8

In [18]:
#5
for i in C:
    
    expr1 = LinExpr(0)
    expr2 = LinExpr(0)
    
    for j in V:
        
            expr1.addTerms(1.0, x[i][j])
    print(expr1)
    if i in C_U:
        for k in U:
            expr2.addTerms(1.0, y[k][i])
    
    print(expr2)
    print("------------------")
    model.addConstr(expr1 + expr2 == 1, "served customer once")
    expr1.clear()
    expr2.clear()
expr1.clear()
expr2.clear()

<gurobi.LinExpr: x1,0 + x1,1 + x1,2 + x1,3 + x1,4 + x1,5 + x1,6 + x1,7 + x1,8 + x1,9>
<gurobi.LinExpr: y0,1 + y1,1>
------------------
<gurobi.LinExpr: x2,0 + x2,1 + x2,2 + x2,3 + x2,4 + x2,5 + x2,6 + x2,7 + x2,8 + x2,9>
<gurobi.LinExpr: y0,2 + y1,2>
------------------
<gurobi.LinExpr: x3,0 + x3,1 + x3,2 + x3,3 + x3,4 + x3,5 + x3,6 + x3,7 + x3,8 + x3,9>
<gurobi.LinExpr: 0.0>
------------------
<gurobi.LinExpr: x4,0 + x4,1 + x4,2 + x4,3 + x4,4 + x4,5 + x4,6 + x4,7 + x4,8 + x4,9>
<gurobi.LinExpr: y0,4 + y1,4>
------------------
<gurobi.LinExpr: x5,0 + x5,1 + x5,2 + x5,3 + x5,4 + x5,5 + x5,6 + x5,7 + x5,8 + x5,9>
<gurobi.LinExpr: y0,5 + y1,5>
------------------
<gurobi.LinExpr: x6,0 + x6,1 + x6,2 + x6,3 + x6,4 + x6,5 + x6,6 + x6,7 + x6,8 + x6,9>
<gurobi.LinExpr: y0,6 + y1,6>
------------------
<gurobi.LinExpr: x7,0 + x7,1 + x7,2 + x7,3 + x7,4 + x7,5 + x7,6 + x7,7 + x7,8 + x7,9>
<gurobi.LinExpr: y0,7 + y1,7>
------------------
<gurobi.LinExpr: x8,0 + x8,1 + x8,2 + x8,3 + x8,4 + x8,5 + x8,6

In [19]:
#6
for i in C:
    expr1 = LinExpr(0)
    expr2 = LinExpr(0)
    
    for j in V:
        
            expr1.addTerms(1.0, x[j][i])
            
    for h in V:
        
            expr2.addTerms(1.0, x[i][h])
    model.addConstr(expr1 == expr2, "flow conservation")
    expr1.clear()
    expr2.clear()
expr1.clear()
expr2.clear()


In [20]:
#get all subtours
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

S = list(powerset(range(1,len(data.cities))))
# The first element of the list is the empty set and the last element is the full set, hence we remove them.
S = S[1:(len(S))]
S = [list(s) for s in S]

print(len(S))
import sys
print(sys.getsizeof(S)/1024/1024," GB")
print(type(S))

    

for s in S:
    
        s.insert(0,0)
S.insert(0,[0])

print(S)
# S = [[0,1,2,3,4]]

511
0.00406646728515625  GB
<class 'list'>
[[0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 1, 5], [0, 1, 6], [0, 1, 7], [0, 1, 8], [0, 1, 9], [0, 2, 3], [0, 2, 4], [0, 2, 5], [0, 2, 6], [0, 2, 7], [0, 2, 8], [0, 2, 9], [0, 3, 4], [0, 3, 5], [0, 3, 6], [0, 3, 7], [0, 3, 8], [0, 3, 9], [0, 4, 5], [0, 4, 6], [0, 4, 7], [0, 4, 8], [0, 4, 9], [0, 5, 6], [0, 5, 7], [0, 5, 8], [0, 5, 9], [0, 6, 7], [0, 6, 8], [0, 6, 9], [0, 7, 8], [0, 7, 9], [0, 8, 9], [0, 1, 2, 3], [0, 1, 2, 4], [0, 1, 2, 5], [0, 1, 2, 6], [0, 1, 2, 7], [0, 1, 2, 8], [0, 1, 2, 9], [0, 1, 3, 4], [0, 1, 3, 5], [0, 1, 3, 6], [0, 1, 3, 7], [0, 1, 3, 8], [0, 1, 3, 9], [0, 1, 4, 5], [0, 1, 4, 6], [0, 1, 4, 7], [0, 1, 4, 8], [0, 1, 4, 9], [0, 1, 5, 6], [0, 1, 5, 7], [0, 1, 5, 8], [0, 1, 5, 9], [0, 1, 6, 7], [0, 1, 6, 8], [0, 1, 6, 9], [0, 1, 7, 8], [0, 1, 7, 9], [0, 1, 8, 9], [0, 2, 3, 4], [0, 2, 3, 5], [0, 2, 3, 6], [0, 2, 3, 7], [0, 2, 3, 8], [0, 2, 3, 9], [0, 2, 

In [22]:
for s in S:
    expr1 = LinExpr(0)
    expr2 = LinExpr(0)
    
    for i in s:
        for j in V:
            if j not in s:
                expr1.addTerms(1.0, x[i][j])
                
    for i in s:
        for k in U:
            if i in C_U:
                expr2.addTerms(1.0, y[k][i])
    model.update()
    print(expr1)
    print(expr2)
    model.addConstr(expr1 + expr2 >= 1)
    expr1.clear()
    expr2.clear()


<gurobi.LinExpr: x0,1 + x0,2 + x0,3 + x0,4 + x0,5 + x0,6 + x0,7 + x0,8 + x0,9>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: x0,2 + x0,3 + x0,4 + x0,5 + x0,6 + x0,7 + x0,8 + x0,9 + x1,2 + x1,3 + x1,4 + x1,5 + x1,6 + x1,7 + x1,8 + x1,9>
<gurobi.LinExpr: y0,1 + y1,1>
<gurobi.LinExpr: x0,1 + x0,3 + x0,4 + x0,5 + x0,6 + x0,7 + x0,8 + x0,9 + x2,1 + x2,3 + x2,4 + x2,5 + x2,6 + x2,7 + x2,8 + x2,9>
<gurobi.LinExpr: y0,2 + y1,2>
<gurobi.LinExpr: x0,1 + x0,2 + x0,4 + x0,5 + x0,6 + x0,7 + x0,8 + x0,9 + x3,1 + x3,2 + x3,4 + x3,5 + x3,6 + x3,7 + x3,8 + x3,9>
<gurobi.LinExpr: 0.0>
<gurobi.LinExpr: x0,1 + x0,2 + x0,3 + x0,5 + x0,6 + x0,7 + x0,8 + x0,9 + x4,1 + x4,2 + x4,3 + x4,5 + x4,6 + x4,7 + x4,8 + x4,9>
<gurobi.LinExpr: y0,4 + y1,4>
<gurobi.LinExpr: x0,1 + x0,2 + x0,3 + x0,4 + x0,6 + x0,7 + x0,8 + x0,9 + x5,1 + x5,2 + x5,3 + x5,4 + x5,6 + x5,7 + x5,8 + x5,9>
<gurobi.LinExpr: y0,5 + y1,5>
<gurobi.LinExpr: x0,1 + x0,2 + x0,3 + x0,4 + x0,5 + x0,7 + x0,8 + x0,9 + x6,1 + x6,2 + x6,3 + x6,4 + x6,5 + x6,7 + x6

In [23]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 560 rows, 119 columns and 16367 nonzeros
Model fingerprint: 0x5ba8334b
Variable types: 1 continuous, 118 integer (118 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 19 rows and 3 columns
Presolve time: 0.02s
Presolved: 541 rows, 116 columns, 16066 nonzeros
Variable types: 0 continuous, 116 integer (115 binary)

Root relaxation: objective 5.700000e+00, 56 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.70000    0    6          -    5.70000      -     -    0s
     0     0    8.30000    0    8          -    8.30000      -     -    0s
     0     0    8.800

In [ ]:
model.ObjVal

In [ ]:
model.printAttr('X')

In [ ]:
model.getVars()